##Installation of the libraries

In [41]:
!pip install -q transformers

In [42]:
!pip install biopython

In [43]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Torch optimization.

##All libraries needed for training

In [45]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting rid of the sequences which are outside of the threshold (64-128).

In [47]:
pdbl = PDBList()

# Define the source directory containing the files
source_dir = '/content/source'

# Define the target directory where the filtered files will be moved
target_dir = '/content/target'

# Ensure the target directory exists, if not, create it
#os.makedirs(target_dir, exist_ok=True)



# Define a function to determine if a file meets the filtering criteria
def filter_criteria(file_path,file_model):
    #pdbl.retrieve_pdb_file("3goe", file_format='mmCif', pdir=".")
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
    # import the needed class
    # instantiate the class to prepare the parser
    cif_parser = MMCIFParser()
    #structure = cif_parser.get_structure("3goe", "3goe.cif")
    structure = cif_parser.get_structure(file_model, file_path)
    model0 = structure[0]

    #model0 = structure[1] - error due to there only being one model
    chain_A = model0['A']
    #structure = cif_parser.get_structure("3goe", "3goe.cif")
    print(structure[0]["A"])
    if 64 <= len(structure[0]["A"]) <= 128:
        shutil.move(file_path, target_dir)
        print(f"Moved file: {file_path} to {target_dir}")

for item in os.listdir(source_dir):
    item_path = os.path.join(source_dir, item)
    item_model = item.replace(".cif","")
    filter_criteria(item_path,item_model)




Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired structure doesn't exist
<Chain id=A>
Desired st

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [48]:
file_path = "/content/target/AF-A0A1D8PD42-F1-model_v4.cif"
file_model = "AF-A0A1D8PD42-F1-model_v4"
pdbl = PDBList()
pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

seq = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    seq.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(seq))

Desired structure doesn't exist
MSSSNTDNQYPKYINDTTPPTITLKEYDNASWASTTCLDHNPIKNQYIVVVMENPNQIVAIIDQQDNMILDILFKNAHDAHSKQEYSTK


Calculating the angles for the given sequence

In [49]:
#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file

AF-A0A1D8PD42-F1-model_v4 A coordinates for 0 dihedra updated in 0 iterations


Putting angles in a matrix.

In [50]:
phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)
angles = np.vstack((psi,phi))

Changing sequence for to be used in the Prot-Bert embedding.

In [51]:
seq_example =  ' '.join(seq)
seq_example

'M S S S N T D N Q Y P K Y I N D T T P P T I T L K E Y D N A S W A S T T C L D H N P I K N Q Y I V V V M E N P N Q I V A I I D Q Q D N M I L D I L F K N A H D A H S K Q E Y S T K'

Data to be given to the transformer.

In [52]:
data = [seq_example,angles]

##Embedding space creation (using Prot-Bert)

In [53]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
seq_example = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq_example, return_tensors='pt')
output = model(**encoded_input)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Getting the dimensions of the embedding space.

In [54]:
N , D = output.last_hidden_state.size()[1], output.last_hidden_state.size()[2]
embedding_prot_bert = output.last_hidden_state

##Embedding layer

In [55]:
class Embedding(nn.Module):
    """
    Embedding layer that combines tokens embeddings with positional encodings
    """
    def __init__(self, n_token, embedding_dim):
        """
        :param n_token: total number of unique token
        :param embedding_dim: dimension of the embedding space
        """
        super(Embedding, self).__init__()
        # Token embedding layer
        self.token_embedding= nn.Embedding(n_token, embedding_dim)
        # positional encoding layer
        self.positional_encoding= PositionalEncoding(embedding_dim)

    def forward(self, input_seq):
        """
        Forward pass for the embedding layer.

        :param input_seq: Input sequence of token indices
        :return: Combined embeddings with positional encodings
        """
        #token_embeddings= self.token_embedding(input_seq)
        #positional_encodings= self.positional_encoding(input_seq)
        #embeddings= token_embeddings + positional_encodings
        embeddings = embedding_prot_bert# Using the prot-bert embedding directly
        return embeddings

class PositionalEncoding(nn.Module):
    """
    Positional encoding layer to add positional information to token embeddings
    """
    def __init__(self, embedding_dim, max_len= 100):
        """
        Initializes the positional encoding layer.

        :param embedding_dim: Dimension of the embedding space
        :param max_len: Maximum length of the sequences to be encoded
        """
        super(PositionalEncoding, self).__init__()
        #create a matrix of position (0 to max_len-1) with shape [max_len, 1]
        position= torch.arange(0, max_len).unsqueeze(1)
        #compute the division term for the position encodings
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * -(np.log(10000.0) / embedding_dim))
        #initialize a matrix for positional encodings with shape [max_len, 1, embedding_dim]
        pe = torch.zeros(max_len, 1, embedding_dim)
        #apply sine to even indices in the array
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        #apply cosine to odd indices in the array
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, input_seq):
        """
        forward pass for the positional encoding layer
        :param input_seq: input tensor of shape [batch_size, seq_len]
        :return: positional encodings corresponding to the input sequence length
        """
        #get the positional encodings up to the length of the input sequence
        return self.pe[:input_seq.size(1), :].transpose(0, 1)


##Single-head self unmasked attention layer

In [56]:
class SingleHeadAttention(nn.Module):
    def __init__(
        self,
        inp_ch: int,
        inp_seq: int,
        key_dim: list,
        que_dim: list,
        out_ch: int,
        out_seq: int,
        d_k: int,
        device,
        verbose=False) -> None:
        """
        Initializes the SingleHeadAttention module.

        :param inp_ch: Number of input channels (N)
        :param inp_seq: Input sequence length (T)
        :param key_dim: List containing dimensions for the key projection matrix (k, s)
        :param que_dim: List containing dimensions for the query projection matrix (k, u)
        :param out_ch: Number of output channels (m)
        :param out_seq: Output sequence length (o)
        :param d_k: Dimension for scaling in dot-product attention
        :param device: Device to perform computations on (CPU/GPU)
        :param verbose: Flag for printing intermediate shapes for debugging
        """
        super(SingleHeadAttention, self).__init__()

        self.N = inp_ch #number of input channels
        self.T = inp_seq #input sequence length
        self.k, self.s = key_dim #dimensions for the key projection matrix
        self.k, self.u = que_dim #dimensions for the query projectoin matrix
        self.m = out_ch #number of output channels
        self.o = out_seq #output sequence length
        self.d_k = d_k #dimension for scaling in dot-product attention
        self.device = device #device to perform computations on

        #initialize learnable weight parameters for q, k, and v matrices
        self.w_q = nn.Parameter(torch.randn([self.s, self.T]))
        self.w_k = nn.Parameter(torch.randn([self.u, self.o]))
        self.w_v = nn.Parameter(torch.randn([self.m, self.N]))
        self.verbos = verbose #flag for printing intermediate shapes for debugging


    def forward(self, Q, K, V):
        """
        Forward pass for the SingleHeadAttention module.

        :param Q: Query tensor
        :param K: Key tensor
        :param V: Value tensor
        :return: Tuple of (context, attention weights)
        """
        #project the input tensors q, k, and v using the corresponding weight matrices
        Q = torch.matmul(Q, self.w_q.to(self.device))
        K = torch.matmul(K, self.w_k.to(self.device))
        V = torch.matmul(V, self.w_v.to(self.device))

        #compute the attention scores by by scaling the dot product of q and k
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k)
        #apply the softmax function to obtain the attention weights
        attn = torch.softmax(scores, dim=-1)
        # Compute the context vector as the weighted sum of the values V
        context = torch.matmul(attn, V)
        return context, attn


In [57]:
class TransformerEncoder(nn.Module):
    def __init__(
        self,
        n_token,
        embedding_dim,
        num_heads,
        feed_forward_dim,
        num_layers,
        dropout_rate=0.1,
        max_len= 100
    ):
        super(TransformerEncoder, self).__init__()
        #stack multiple transformer encoder layers
        #each layer is an instance of the TransformerEncoderLayer class
        self.layers = nn.ModuleList([TransformerEncoderLayer(n_token, embedding_dim, num_heads, feed_forward_dim, dropout_rate, max_len) for _ in range(num_layers)])

    def forward(self, input_seq):
        out = input_seq #initialize the output as the input sequence
        #pass the input through each transformer encoder layer sequentially
        for layer in self.layers:
            out = layer(out) #update the output after each layer
        return out

##Encoder with attention and 2 layer FFNN

In [58]:
class TransformerEncoderLayer(nn.Module):
    def __init__(
        self,
        n_token,
        embedding_dim,
        num_heads,
        feed_forward_dim,
        dropout_rate= 0.1,
        max_len= 100):
        super(TransformerEncoderLayer, self).__init__()
        #Embedding layer: maps each token to a dense vector of fixed size (embedding dim)
        self.embedding= nn.Embedding(n_token, embedding_dim)
        #positional encoding layer: adds positional information to the token embeddings
        self.positional_encoding = PositionalEncoding(embedding_dim, max_len)
        #single head self attention layer: computee attention scores and context vectors
        self.self_attention = SingleHeadAttention(
            inp_ch=embedding_dim,
            inp_seq=max_len,
            key_dim=[embedding_dim // num_heads, embedding_dim],
            que_dim=[embedding_dim // num_heads, embedding_dim],
            out_ch=embedding_dim,
            out_seq=max_len,
            d_k=embedding_dim // num_heads,
            device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
            verbose=False
        )
        #layer normalization layers: normalize the outputs of each sub-layer
        self.layer_norm1= nn.LayerNorm(embedding_dim)
        self.layer_norm2= nn.LayerNorm(embedding_dim)

        #feed-forward network: three linear layers with ReLU activation in between
        self.feed_forward = nn.Sequential(
            nn.Linear(embedding_dim, feed_forward_dim),
            nn.ReLU(),
            nn.Linear(feed_forward_dim, feed_forward_dim),
            nn.ReLU(),
            nn.Linear(feed_forward_dim, embedding_dim)
        )
        #dropout layer: randomly zeroes some of the elements of the input tensor with probability dropout_rate
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_seq):
        #compute token embeddings
        token_embeddings = self.embedding(input_seq)
        #compute positional encodings
        positional_encoding= self.positional_encoding(input_seq)
        embeddings = token_embeddings + positional_encoding

        #self_attention
        attn_output, _ = self.self_attention(embeddings, embeddings, embeddings)
        #apply dropout to attention output
        attn_output = self.dropout(attn_output)
        #add residual connection and apply layer normalizaiton
        out1 = self.layer_norm1(embeddings + attn_output)

        #apply feed-forward network
        ffn_output = self.feed_forward(out1)
        #apply dropout to feed-foward output
        ffn_output = self.dropout(ffn_output)
        #add residual connection and apply layer normalization
        out2 = self.layer_norm2(out1 + ffn_output)

        return out2

In [68]:
class TransformerEncoder(nn.Module):
    def __init__(
        self,
        n_token,
        embedding_dim,
        output_dim,
        num_heads,
        feed_forward_dim,
        num_layers=1,
        dropout_rate=0.1,
        max_len= 100
    ):
        super(TransformerEncoder, self).__init__()
        #stack multiple transformer encoder layers
        #each layer is an instance of the TransformerEncoderLayer class
        #self.layers = nn.ModuleList([TransformerEncoderLayer(n_token, embedding_dim, output_dim, num_heads, feed_forward_dim, dropout_rate, max_len) for _ in range(num_layers)])

    def forward(self, input_seq):
        out = input_seq #initialize the output as the input sequence
        #pass the input through each transformer encoder layer sequentially
        for layer in self.layers:
            out = layer(out) #update the output after each layer
        return out

##Training

In [69]:
model = TransformerEncoder(N, D, 2, 1, 200)
model = model.to(device)
num_epochs = 100

#define the loss function and optimizer
criterion = nn.MSELoss()#nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameter(), lr=0.001) #model.parameter?

def train(model, data, criterion, optimizer, device):
    model.train()
    total_loss = 0

    for input_seq, target_seq in data:
        input_seq, target_seq = input_seq.to(device), target_seq.to(device)

        #clear gradients w.r.t. parameters
        optimizer.zero_grad()

        output_seq = model(input_seq)

        #reshape output and target to calculate loss
        output_seq = output_seq.view(-1, N)
        target_seq = target_seq.view(-1)

        #calculate loss: softmax --> cross entropy loss
        loss = criterion(output_seq, target_seq)

        #backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss +=loss.item()

    return total_loss / len(data)

for epoch in range(num_epochs):
    epoch_loss = train(model, data, criterion, optimizer, device)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

AttributeError: 'TransformerEncoder' object has no attribute 'parameter'